In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [4]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from ),
    end_time=(fetch_data_to),
)

ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-06 21:41:16,748 INFO: Initializing external client
2025-03-06 21:41:16,749 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 21:41:17,658 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215708
Fetching data from 2025-02-06 02:41:16.745015+00:00 to 2025-03-07 01:41:16.745015+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.48s) 


In [5]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-06 03:00:00+00:00,2,0
1,2025-02-06 04:00:00+00:00,2,0
2,2025-02-06 05:00:00+00:00,2,0
3,2025-02-06 06:00:00+00:00,2,0
4,2025-02-06 07:00:00+00:00,2,0
...,...,...,...
175135,2025-03-06 21:00:00+00:00,263,99
175136,2025-03-06 22:00:00+00:00,263,90
175137,2025-03-06 23:00:00+00:00,263,48
175138,2025-03-07 00:00:00+00:00,263,43


In [6]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175140 entries, 0 to 175139
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         175140 non-null  datetime64[ns, Etc/UTC]
 1   pickup_location_id  175140 non-null  int32                  
 2   rides               175140 non-null  int32                  
dtypes: datetime64[ns, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [7]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175140 entries, 0 to 175139
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         175140 non-null  datetime64[ns]
 1   pickup_location_id  175140 non-null  int32         
 2   rides               175140 non-null  int32         
dtypes: datetime64[ns](1), int32(2)
memory usage: 4.0 MB


In [9]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [10]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,32,85,229,145,155,22,116,5,111,80,...,111,123,127,108,55,97,137,108,263,2025-03-05 17:00:00
1,2,0,0,0,0,3,0,0,1,0,...,8,0,1,5,1,1,0,0,255,2025-03-05 10:00:00
2,0,0,0,12,0,0,2,0,0,2,...,1,3,3,2,1,0,2,1,112,2025-02-15 06:00:00
3,0,2,1,2,0,1,0,0,1,1,...,0,0,0,0,0,0,0,2,197,2025-03-03 12:00:00
4,0,0,0,0,2,0,0,1,0,1,...,0,0,1,0,0,0,0,0,119,2025-02-13 01:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,253,2025-02-21 16:00:00
248,2,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,254,2025-03-05 15:00:00
249,1,173,130,4,423,148,41,152,141,220,...,166,182,10,311,13,225,29,14,142,2025-02-24 05:00:00
250,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,136,2025-02-25 09:00:00


In [11]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-06 21:41:29,817 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 21:41:29,841 INFO: Initializing external client
2025-03-06 21:41:29,844 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 21:41:30,632 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215708
Fetching data from 2025-02-06 02:41:29.817607+00:00 to 2025-03-07 01:41:29.817607+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.42s) 


In [12]:
current_date

Timestamp('2025-03-07 02:41:29.817607+0000', tz='Etc/UTC')

In [13]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-06 03:00:00
1,0,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,3,2025-03-06 03:00:00
2,0,1,0,1,5,7,2,4,4,1,...,17,9,12,18,6,2,2,0,4,2025-03-06 03:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-06 03:00:00
4,0,0,1,2,0,4,3,2,5,4,...,5,3,1,1,1,0,1,0,7,2025-03-06 03:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,0,1,0,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,259,2025-03-06 03:00:00
248,2,0,0,0,1,0,0,0,0,0,...,1,1,0,1,0,2,0,1,260,2025-03-06 03:00:00
249,0,2,2,2,7,7,16,27,17,31,...,78,57,48,34,12,0,3,1,261,2025-03-06 03:00:00
250,0,6,17,71,157,174,161,112,89,110,...,98,56,40,42,9,2,2,1,262,2025-03-06 03:00:00


In [14]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-06 21:41:41,726 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 21:41:41,735 INFO: Initializing external client
2025-03-06 21:41:41,736 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 21:41:42,453 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215708

In [15]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


In [16]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,0.0
3,6,0.0
4,7,0.0
...,...,...
247,259,0.0
248,260,0.0
249,261,1.0
250,262,2.0


In [17]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([246,  48,  79, 249, 230, 148,  68, 231, 125, 132])